In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Load the dataset
file_path = 'C:/Users/shume/Downloads/CAP_Data/final_dataset.csv'
data = pd.read_csv(file_path)

# Display basic information about the dataset and the first few rows
data.info(), data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252135 entries, 0 to 252134
Columns: 198 entries, SK_ID_CURR to AVG_APPLICATION_AMOUNT
dtypes: float64(141), int64(41), object(16)
memory usage: 380.9+ MB


(None,
    SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
 0      100002       1         Cash loans           M            N   
 1      100003       0         Cash loans           F            N   
 2      100004       0    Revolving loans           M            Y   
 3      100006       0         Cash loans           F            N   
 4      100007       0         Cash loans           M            N   
 
   FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
 0               Y             0          202500.0    406597.5      24700.5   
 1               N             0          270000.0   1293502.5      35698.5   
 2               Y             0           67500.0    135000.0       6750.0   
 3               Y             0          135000.0    312682.5      29686.5   
 4               Y             0          121500.0    513000.0      21865.5   
 
    ...  DAYS_INSTALMENT_MEAN_y DAYS_INSTALMENT_MAX_y DAYS_INSTALMENT_MIN_y  \
 0  ...         

Check for duplicates in the SK_ID_CURR column to ensure that each key is unique.
Look for any missing values in this key column, which might cause issues during data integration or matching.

In [2]:
# Checking for duplicates and missing values in the SK_ID_CURR column
duplicate_keys = data['SK_ID_CURR'].duplicated().sum()
missing_keys = data['SK_ID_CURR'].isna().sum()
print('Duplicate SK_ID_CURR:', duplicate_keys)
print('Missing SK_ID_CURR:', missing_keys)

Duplicate SK_ID_CURR: 0
Missing SK_ID_CURR: 0


To ensure that the features derived from external datasets like "bureau" and "previous_application" meaningfully contribute to your model, we can perform a few analyses:

Assess the sparsity of these features to determine how much missing data each contains.
Generate summary statistics for numerical features to get an overview of their distributions and identify any potential outliers or unusual patterns.
Examine categorical features for the number of unique categories and the distribution of values within these categories.
Consider strategies for dealing with missing data, such as imputation, creating binary indicators for missing data, or excluding features with excessive missingness if they do not add predictive value.

In [6]:
# Selecting columns from 123 onwards and calculating sparsity
external_data_columns = data.columns[123:]
missing_data_ratios = data[external_data_columns].isna().mean().sort_values(ascending=False)
numerical_columns = data[external_data_columns].select_dtypes(include=['number']).columns  # Simplified to 'number'
categorical_columns = data[external_data_columns].select_dtypes(include=['object']).columns

# Displaying sparsity
print('Top 5 Missing Data Ratios:')
print(missing_data_ratios.head())
print('Numerical Columns:')
print(numerical_columns)
print('Categorical Columns:')
print(categorical_columns)

Top 5 Missing Data Ratios:
CNT_DRAWINGS_ATM_CURRENT_MEAN_x    0.79198
AMT_DRAWINGS_ATM_CURRENT_MEAN_y    0.79198
AMT_DRAWINGS_ATM_CURRENT_MEAN_x    0.79198
CNT_DRAWINGS_ATM_CURRENT_MEAN_y    0.79198
CNT_DRAWINGS_CURRENT_SUM_x         0.70561
dtype: float64
Numerical Columns:
Index(['YEARS_EMPLOYED', 'CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT',
       'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT', 'BUREAU_LOAN_COUNT_x',
       'AVG_CREDIT_SUM_x', 'PREVIOUS_APPLICATION_COUNT_x',
       'AVG_APPLICATION_AMOUNT_x', 'AMT_BALANCE_MEAN_x', 'AMT_BALANCE_MAX_x',
       'AMT_BALANCE_MIN_x', 'AMT_BALANCE_SUM_x',
       'AMT_CREDIT_LIMIT_ACTUAL_MEAN_x', 'AMT_CREDIT_LIMIT_ACTUAL_MAX_x',
       'AMT_DRAWINGS_ATM_CURRENT_MEAN_x', 'AMT_DRAWINGS_ATM_CURRENT_SUM_x',
       'AMT_DRAWINGS_CURRENT_MEAN_x', 'AMT_DRAWINGS_CURRENT_SUM_x',
       'AMT_PAYMENT_TOTAL_CURRENT_MEAN_x', 'AMT_PAYMENT_TOTAL_CURRENT_SUM_x',
       'CNT_DRAWINGS_ATM_CURRENT_MEAN_x', 'CNT_DRAWINGS_ATM_CURRENT_SUM_x',
       'CNT_DRAWINGS

In [7]:
# Summary statistics for the numerical columns
summary_statistics = data[numerical_columns].describe().transpose()
print(summary_statistics)

                               count           mean            std  \
YEARS_EMPLOYED              252135.0       6.527552       6.402080   
CREDIT_INCOME_PERCENT       252135.0       3.857996       2.572014   
ANNUITY_INCOME_PERCENT      252123.0       0.177266       0.090983   
CREDIT_TERM                 252123.0       0.053892       0.022521   
DAYS_EMPLOYED_PERCENT       252135.0       0.156863       0.133548   
...                              ...            ...            ...   
DAYS_ENTRY_PAYMENT_MIN_y    238829.0   -1585.284061     910.505471   
BUREAU_LOAN_COUNT           216475.0       5.583236       4.380794   
AVG_CREDIT_SUM              216474.0  410034.843870  968767.042907   
PREVIOUS_APPLICATION_COUNT  238466.0       4.781013       4.049168   
AVG_APPLICATION_AMOUNT      238466.0  153068.174776  156237.954134   

                                    min            25%            50%  \
YEARS_EMPLOYED                 0.002738       2.099932       4.511978   
CREDIT_INCOME

Let's categorize the columns based on the percentage of missing data and suggest appropriate strategies for each category:

Low Missingness (Less than 20%): Impute using simple methods like mean, median, or mode.
Moderate Missingness (20% to 50%): Consider more sophisticated imputation methods or using placeholders if the feature is important.
High Missingness (More than 50%): Evaluate the necessity of the feature. Consider excluding it or using model-based imputation if the feature is crucial.

In [8]:
# Categorizing columns based on the percentage of missing data
thresholds = {
    'low': (0, 0.20),
    'moderate': (0.20, 0.50),
    'high': (0.50, 1.0)
}
missingness_categories = {
    k: missing_data_ratios[(missing_data_ratios > v[0]) & (missing_data_ratios <= v[1])]
    for k, v in thresholds.items()
}

# Display missingness categories
for category, values in missingness_categories.items():
    print(f'{category.capitalize()} Missingness: {len(values)} columns')

# Count the number of columns in each category
column_counts = {category: len(cols) for category, cols in missingness_categories.items()}

# Output the number of columns in each category
print('Column counts by missingness category:', column_counts)

Low Missingness: 40 columns
Moderate Missingness: 0 columns
High Missingness: 32 columns
Column counts by missingness category: {'low': 40, 'moderate': 0, 'high': 32}


In [10]:
from sklearn.impute import SimpleImputer

# Check and impute for low missingness
if 'low' in missingness_categories and not missingness_categories['low'].empty:
    low_missing_numerical = [col for col in missingness_categories['low'].index if col in numerical_columns]
    if low_missing_numerical:
        imputer = SimpleImputer(strategy='mean')
        data[low_missing_numerical] = imputer.fit_transform(data[low_missing_numerical])
    else:
        print("No numerical columns with low missingness to impute.")

# Drop high missingness columns
if 'high' in missingness_categories and not missingness_categories['high'].empty:
    high_missing_columns = list(missingness_categories['high'].index)
    data.drop(columns=high_missing_columns, inplace=True)
else:
    print("No columns with high missingness to drop.")

# Optionally handle moderate missingness if it exists and is not empty
if 'moderate' in missingness_categories and not missingness_categories['moderate'].empty:
    moderate_missing_numerical = [col for col in missingness_categories['moderate'].index if col in numerical_columns]
    if moderate_missing_numerical:
        imputer_moderate = SimpleImputer(strategy='median')
        data[moderate_missing_numerical] = imputer_moderate.fit_transform(data[moderate_missing_numerical])
    else:
        print("No numerical columns with moderate missingness to impute.")

# Check the dataframe structure after modifications
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252135 entries, 0 to 252134
Columns: 166 entries, SK_ID_CURR to AVG_APPLICATION_AMOUNT
dtypes: float64(109), int64(41), object(16)
memory usage: 319.3+ MB
None


In [13]:
from sklearn.impute import SimpleImputer

# Identify numerical and categorical columns with low missingness
low_missing_categorical = [col for col in missingness_categories['low'].index if col in categorical_columns]

# Imputing numerical data (already handled)
# Now handling categorical data
if low_missing_categorical:
    imputer_categorical = SimpleImputer(strategy='most_frequent')  # or strategy='constant', fill_value='Unknown'
    data[low_missing_categorical] = imputer_categorical.fit_transform(data[low_missing_categorical])

# If you previously dropped high missingness columns and now want to handle categorical data separately
# Redefine high missing columns excluding categorical ones, if you wish to keep some categorical columns with high missingness


In [14]:
# Rechecking and separating the categorical columns with low missingness for imputation
low_missing_categorical = [col for col in missingness_categories['low'].index if col in data.select_dtypes(include=['object']).columns]

# Imputing categorical data using the most frequent strategy
if low_missing_categorical:
    imputer_categorical = SimpleImputer(strategy='most_frequent')
    data[low_missing_categorical] = imputer_categorical.fit_transform(data[low_missing_categorical])

# Confirm the successful imputation and check if any categorical columns still have missing values
categorical_missing_after = data[low_missing_categorical].isna().sum()
low_missing_categorical, categorical_missing_after


([], Series([], dtype: float64))

In [16]:
# Checking for remaining NaN values across all columns
remaining_nans = data.isna().sum().sort_values(ascending=False)
remaining_nans = remaining_nans[remaining_nans > 0]

remaining_nans


COMMONAREA_AVG              175886
COMMONAREA_MODE             175886
COMMONAREA_MEDI             175886
NONLIVINGAPARTMENTS_AVG     174759
NONLIVINGAPARTMENTS_MODE    174759
                             ...  
EXT_SOURCE_2                   504
AMT_GOODS_PRICE                256
AMT_ANNUITY                     12
CNT_FAM_MEMBERS                  2
DAYS_LAST_PHONE_CHANGE           1
Length: 67, dtype: int64

In [17]:
from sklearn.impute import SimpleImputer

# Setting up imputers for numerical and categorical data
numerical_imputer = SimpleImputer(strategy='median')  # Median imputation for numerical data
categorical_imputer = SimpleImputer(strategy='most_frequent')  # Most frequent for categorical data

# Separating numerical and categorical columns with missing values
numerical_columns_with_nans = data[remaining_nans.index].select_dtypes(include=['number']).columns
categorical_columns_with_nans = data[remaining_nans.index].select_dtypes(include=['object']).columns

# Applying the imputation
if not numerical_columns_with_nans.empty:
    data[numerical_columns_with_nans] = numerical_imputer.fit_transform(data[numerical_columns_with_nans])

if not categorical_columns_with_nans.empty:
    data[categorical_columns_with_nans] = categorical_imputer.fit_transform(data[categorical_columns_with_nans])

# Check again for any remaining NaNs to ensure all have been handled
final_check_nans = data.isna().sum().sum()
final_check_nans


0

In [18]:
# Double-checking the entire dataset for any remaining NaN values
overall_nans = data.isna().sum().sort_values(ascending=False)
remaining_nans = overall_nans[overall_nans > 0]

# Re-impute if necessary
if not remaining_nans.empty:
    # Using median imputation for any remaining numerical NaNs
    remaining_numerical_columns = data[remaining_nans.index].select_dtypes(include=['number']).columns
    if not remaining_numerical_columns.empty:
        data[remaining_numerical_columns] = numerical_imputer.fit_transform(data[remaining_numerical_columns])

# Rechecking for NaNs after re-imputation
final_nan_check = data.isna().sum().sum()

# Preparing features and target again
X = data.drop('TARGET', axis=1)
y = data['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Recheck for NaNs in X specifically
nan_in_X = X.isna().sum().sum()
final_nan_check, nan_in_X


(0, 0)

In [20]:
# Checking the data types to identify any categorical columns mistakenly left as 'object' type
categorical_check = data.select_dtypes(include=['object'])
categorical_columns = categorical_check.columns

# If there are any categorical columns, we will apply Label Encoding
if not categorical_columns.empty:
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    for col in categorical_columns:
        data[col] = le.fit_transform(data[col].astype(str))

# Verifying the transformation
transformed_categorical_check = data[categorical_columns].dtypes

transformed_categorical_check


NAME_CONTRACT_TYPE            int32
CODE_GENDER                   int32
FLAG_OWN_CAR                  int32
FLAG_OWN_REALTY               int32
NAME_TYPE_SUITE               int32
NAME_INCOME_TYPE              int32
NAME_EDUCATION_TYPE           int32
NAME_FAMILY_STATUS            int32
NAME_HOUSING_TYPE             int32
OCCUPATION_TYPE               int32
WEEKDAY_APPR_PROCESS_START    int32
ORGANIZATION_TYPE             int32
FONDKAPREMONT_MODE            int32
HOUSETYPE_MODE                int32
WALLSMATERIAL_MODE            int32
EMERGENCYSTATE_MODE           int32
dtype: object

In [21]:
# Re-preparing the feature set and target after ensuring all data is numeric
X = data.drop('TARGET', axis=1)
y = data['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Retraining the Random Forest model with all data correctly encoded
model.fit(X_train, y_train)

# Predicting the test set
predictions = model.predict(X_test)

# Evaluating the model
classification_report_result = classification_report(y_test, predictions)
auc_roc_score = roc_auc_score(y_test, predictions)

classification_report_result, auc_roc_score


('              precision    recall  f1-score   support\n\n           0       0.91      1.00      0.95     46047\n           1       0.55      0.00      0.00      4380\n\n    accuracy                           0.91     50427\n   macro avg       0.73      0.50      0.48     50427\nweighted avg       0.88      0.91      0.87     50427\n',
 0.5006306391533843)